In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%pylab
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
1
from IPython.display import display, HTML
2
display(HTML("<style>.container { width:95% !important; }</style>"))
3
%pylab
4
from scipy.optimize import curve_fit
5
​
6
import tkinter as tk
7
from tkinter import filedialog
8
​
9
import os
10
​
11
from scipy import signal
12
import scipy.integrate as integrate
13
import matplotlib.ticker as ticker
14
​
15
# %matplotlib qt
16
​
17
%run functions
18
matplotlib.rcParams.update({'font.size': 25})
19
# cm = pylab.get_cmap('tab10')
20
# cm2 = pylab.get_cmap('Set1')
21
cm = pylab.get_cmap('turbo')
import scipy.integrate as integrate
import matplotlib.ticker as ticker

# %matplotlib qt

%run functions
matplotlib.rcParams.update({'font.size': 25})
# cm = pylab.get_cmap('tab10')
# cm2 = pylab.get_cmap('Set1')
cm = pylab.get_cmap('turbo')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.0e6*2*pi # 2.047e6
tauRF = 1/(Omega/2/pi)

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [347]:
n_dt = 500
n_save_trj = 50

In [541]:
def MB_dist_i(T,vi):
    deuxkT = 2*kb*T
    dv = diff(vi)[2]
    return sqrt(m_Ca/(deuxkT*pi))*exp(-m_Ca*vi**2/deuxkT)
def MB_dist(T,v):
    deuxkT = 2*kb*T
    dv = diff(v)[2]
    return sqrt(2/pi)*(m_Ca/(kb*T))**(3/2)*v**2*exp(-m_Ca*v**2/deuxkT)

In [293]:
# To load pos and velocity in time
# stored in a binary file
def load_xv(filename):
    fid = open(filename, 'rb')
    junk = fromfile(fid, int32,1)        # Read record start tag
    n_ions  = fromfile(fid, int32, 1)
    print(n_ions)
    junk = fromfile(fid, int32,1)        # Read record stop tag

    junk = fromfile(fid, int32,1)        # Read record start tag
    allocate  = fromfile(fid, int32, 1)
    print(allocate)
    junk = fromfile(fid, int32,1)        # Read record stop tag
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    j_save_temp  = fromfile(fid, int32, 1);
    print(j_save_temp)
    junk = fromfile(fid, int32,1)        # Read record stop tag

    junk = fromfile(fid, int32,1)        # Read record start tag
    print(junk)
    ri_vi  = fromfile(fid, float64, junk[0]//8);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)

    fid.close
    print(junk[0]//8//6//n_ions[0])

    ri_vi = reshape(ri_vi, (n_ions[0],6, junk[0]//8//6//n_ions[0]),order='F')
    return n_ions, allocate, j_save_temp, ri_vi

# 24576000 = 1024*6*500*8

In [305]:
# load temperature of the cloud as save in .dat
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20230125'
all_subdir = []
all_Temp = []
all_trjN = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'SimuType0' and '.dat' in l:
            all_Temp.append(j[0]+'/'+l)
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
        #print(j[0],'\n')
all_subdir.sort()
all_Temp.sort()
all_xva_3DHarmo.sort()
all_xva_start.sort()
all_trjN.sort()
# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')

In [306]:
for j,k in enumerate(all_subdir):
    str_N  = all_Temp[j].find('_N')
    cond_N = all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
#     outfile = 'Time_and_temp_RFHEAT_N'+ str(cond_N) + '_alphavar_' + str(cond)
    
    data = loadtxt(all_Temp[j])
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    save_xva = load_xyz_init_bin_DP(all_xva_start[j].strip('.bin'))
    alpha = max(sqrt(save_xva[0]**2+save_xva[1]**2))/max(save_xva[2])
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]
    
#     np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2, alpha=alpha) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
#     print(outfile) # , r2_v2_rlim=r2_v2

DC10_RF10


In [309]:
# plot temperature
figname = 'T_vs_time'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.semilogy(time_s*1e3, T_aux_avg ,label=r'$T$',color='C0',lw=2)
# ax1.semilogy(time_s*1e3, T_aux[:,0],color='C2')
# ax1.semilogy(time_s*1e3, T_aux[:,2],color='C9')
ax1.semilogy(time_s[::n_save_trj][1:]*1e3, T_aux_avg[::n_save_trj][1:] ,label=r'$T$',color='C2',marker='P',ls='')
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
# savefig(figname+'1024.eps',dpi=300)

In [304]:
dirname = '/home/adrien/RemoteFS/Rivendel/Simulations/20230125/DC10_RF10/Try00'
filename = '/xva_SimuType0_N01024_Vrf0065_Udc0.1508D+02V_nt1000_TrjN.bin'
n_ions, allocate, j_save_temp, ri_vi = load_xv(dirname+filename)

[1024]
[9935]
[64]
[491520000]
[491520000]
10000


In [317]:
# ri_vi = ri_vi/n_dt

# T = 1.00e-02
for `time_to_plot = 2`

In [620]:
print(j_save_temp)
time_to_plot = 2
print(time_s[::50][time_to_plot])

[64]
0.001050624


In [614]:
# plot ions positions
figname = 'y_vs_x'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.plot(ri_vi[:,0,time_to_plot]*1e6,ri_vi[:,1,time_to_plot]*1e6,label=r'$T$',color='C0',marker='o',ls='')
ax1.set_xlabel('x [µm]')
ax1.set_ylabel('y [µm]')
tight_layout()

# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

In [613]:
# plot pos vs velocity
figname = 'vx_vs_x'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.plot(ri_vi[:,0,time_to_plot]*1e6,ri_vi[:,3,time_to_plot],label=r'$T$',color='C0',marker='o',ls='')
ax1.set_xlabel('x [µm]')
ax1.set_ylabel('$v_x$ [m/s]')
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

In [612]:
# plot pos vs velocity x direction
vi = arange(-5,5,0.5)
T = T_aux[::n_save_trj][time_to_plot+1][0]
figname = 'vx_hist'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.hist(ri_vi[:,3,time_to_plot], bins=arange(-10,10,0.5),ec='k') # bins='auto'
ax1.plot(vi,MB_dist_i(T,vi)*1024*0.5)
ax1.set_xlabel('$v_x$ [m/s]')
print(f'T = {T:.2e} K')
# ax1.set_ylabel('$v_x$ [m/s]')
ax1.set_xlim(-5,5)
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

T = 1.00e-02 K


In [526]:
sum(MB_dist_i(T,vi))

0.9994820403782344

In [610]:
# plot pos vs velocity z direction
vi = arange(-10,10,0.1)
T = T_aux[::n_save_trj][time_to_plot+1][2]
figname = 'vz_hist'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.hist(ri_vi[:,5,time_to_plot], bins=arange(-10,10,0.1),ec='k') # bins='auto'
ax1.plot(vi,MB_dist_i(T,vi)*1024*0.1)
ax1.set_xlabel('$v_z$ [m/s]')
print(f'T = {T:.2e} K')
# ax1.set_ylabel('$v_x$ [m/s]')
ax1.set_xlim(-2.5,2.5)
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

T = 2.18e-03 K


In [573]:
sum(MB_dist_i(T,vi))*0.1

1.0000000000000036

In [621]:
# plot pos vs velocity one direction
v = arange(0,5,0.1)
T = T_aux_avg[::n_save_trj][time_to_plot+1]
figname = 'v_hist'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.hist(sqrt(sum(ri_vi[:,3:,time_to_plot]**2,axis=1)), bins=arange(-10,10,0.1),ec='k') # bins='auto'
ax1.plot(v,MB_dist(T,v)*1024*0.1)
ax1.set_xlabel('$v$ [m/s]')
print(f'T = {T:.2e} K')
# ax1.set_ylabel('$v_x$ [m/s]')
ax1.set_xlim(-1,5)
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

T = 4.42e-03 K


In [515]:
sum(MB_dist(T,v))

1.0000000000000002

# T = 1.00e-02
for `time_to_plot = 58`

In [622]:
print(j_save_temp)
time_to_plot = 58
print(time_s[::50][time_to_plot])

[64]
0.002450624


In [606]:
# plot ions positions
figname = 'y_vs_x'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.plot(ri_vi[:,0,time_to_plot]*1e6,ri_vi[:,1,time_to_plot]*1e6,label=r'$T$',color='C0',marker='o',ls='')
ax1.set_xlabel('x [µm]')
ax1.set_ylabel('y [µm]')
tight_layout()

# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

In [604]:
# plot pos vs velocity
figname = 'vx_vs_x'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.plot(ri_vi[:,0,time_to_plot]*1e6,ri_vi[:,3,time_to_plot],label=r'$T$',color='C0',marker='o',ls='')
ax1.set_xlabel('x [µm]')
ax1.set_ylabel('$v_x$ [m/s]')
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

In [603]:
# plot pos vs velocity x direction
vi = arange(-75,75,5)
T = T_aux[::n_save_trj][time_to_plot+1][0]
figname = 'vx_hist'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.hist(ri_vi[:,3,time_to_plot], bins=arange(-100,100,5),ec='k') # bins='auto'
ax1.plot(vi,MB_dist_i(T,vi)*1024*5)
ax1.set_xlabel('$v_x$ [m/s]')
print(f'T = {T:.2e} K')
# ax1.set_ylabel('$v_x$ [m/s]')
ax1.set_xlim(-90,90)
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

T = 2.85e+00 K


In [602]:
# plot pos vs velocity z direction
vi = arange(-100,100,5)
T = T_aux[::n_save_trj][time_to_plot+1][2]
figname = 'vz_hist'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.hist(ri_vi[:,5,time_to_plot], bins=arange(-100,100,5),ec='k') # bins='auto'
ax1.plot(vi,MB_dist_i(T,vi)*1024*5)
ax1.set_xlabel('$v_z$ [m/s]')
print(f'T = {T:.2e} K')
# ax1.set_ylabel('$v_x$ [m/s]')
ax1.set_xlim(-100,100)
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

T = 4.02e+00 K


In [623]:
# plot pos vs velocity one direction
v = arange(0,250,5)
T = T_aux_avg[::n_save_trj][time_to_plot+1]
figname = 'v_hist'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.hist(sqrt(sum(ri_vi[:,3:,time_to_plot]**2,axis=1)), bins=arange(-100,100,5),ec='k') # bins='auto'
ax1.plot(v,MB_dist(T,v)*1024*5)
ax1.set_xlabel('$v$ [m/s]')
print(f'T = {T:.2e} K')
# ax1.set_ylabel('$v_x$ [m/s]')
ax1.set_xlim(-1,100)
tight_layout()
# subplots_adjust(top=0.93,
# bottom=0.21)
savefig(f'{figname}_{T:.2e}K_1024.eps',dpi=300)

T = 3.18e+00 K
